In [1]:
## For debbugging
%load_ext autoreload

In [2]:
## for debugging:
%aimport mlp.keyword
%aimport mlp.tools
%autoreload 1
from mlp.keyword import *

In [3]:
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
from tqdm import tqdm_notebook
%matplotlib inline


In [4]:
data_directory = os.path.join('.', 'tutorial_data')

nl_col = {
    "Description": 5,
#     "Resolution": 6
    "Resolution": 248
}

meta_col = {
    "AssetID": 6,
    "Location": 7,
    "Assigned": 10,
    "Reported": 43,
    "SolveDate": 11,
    "ReportDate": 44
}

kex = KeywordExtractor('hvac_mslips_20170720.xlsx',
                      nlp_cols=nl_col,
                      meta_cols = meta_col,
                      special_replace={'Action Taken:': ''},
                      wdir=data_directory)

{'Action Taken:': ''}


In [23]:
# what we extracted...we'll be using 'RawText'
kex.df.head()

,Description,AssetID,Location,Assigned,SolveDate,Reported,ReportDate,Resolution,RawText
0,CHARGE ALL M/S AND PM'S TO THIS ORDER,NaN,GAITHER,FEDERLIN,2009-04-02 14:15:38,LEM801,2009-02-12 14:27:33,"for site , other that AML Action Taken:Complete.",charge m_s pm order aml time charge cover pm '...
1,CHARGE ALL M/S AND PM'S TO THIS ORDER FOR THE AML,NaN,215-1,FEDERLIN,2009-04-02 14:15:39,LEM801,2009-02-12 14:44:55,Action Taken: Time charged to cover PM's as ne...,adjust repai replace freeze_stat reset poc_ed
2,"ADJUST,REPAI OR REPLACE FREEZE STAT",301 HV-4,301-B184,FEDERLIN,2009-04-02 14:15:41,HWRJR,2009-02-15 23:25:01,Action Taken:Reset POC Ed,reset adjust repair freeze_stat restart poc_ed
3,"RESET,ADJUST OR REPAIR FREEZE STAT",304 ACU-1,304-200,FEDERLIN,2009-04-02 14:15:42,HWRJR,2009-02-17 04:20:08,Action Taken: restart POC Ed,rset adjust repair freeze_stat reset_freezestat
4,"RSET,ADJUST OR REPAIR FREEZE STAT",220 ACU-8,220-400,FEDERLIN,2009-06-25 08:17:33,HWRJR,2009-02-17 04:22:38,reset freezestat,reset adjust repair freeze_stat reset_alarm hv...


In [16]:
# use this to make a properly formatted thesaurus file for human-annotation
kex.gen_vocab('my_vocab.csv', notes=True)  # rename to match e.g. hvac_vocab.csv

In [17]:
# "fit" a thesaurus to the data
kex.fit(vocab='hvac_vocab.csv')
kex.vocab.head()

,NE,alias,note
token,,,
room,I,room,NaN
replace,S,replace,NaN
find,S,find,NaN
check,S,check,NaN
action_take,X,action_take,NaN


In [20]:
# tag the data, saving to a new DF (instead of internally)
df_pred = kex.transform(save=False)
df_pred.head()

100%|██████████| 16985/16985 [00:41<00:00, 406.63it/s]


,RawText,Items,Problem,Solution,UK_tok
0,charge m_s pm order aml time charge cover pm '...,,,"order, preventative_maintenance","charge, 's, time, time charge, aml, m_s"
1,adjust repai replace freeze_stat reset poc_ed,freeze_stat,,"reset, adjust, replace","repai, poc_ed"
2,reset adjust repair freeze_stat restart poc_ed,freeze_stat,,"reset, adjust, restart, repair",poc_ed
3,rset adjust repair freeze_stat reset_freezestat,freeze_stat,,"adjust, repair",reset_freezestat
4,reset adjust repair freeze_stat reset_alarm hv...,freeze_stat,,"reset, adjust, repair","hv-3, hv-2"


In [18]:
# tag-complete work-orders?
print(df_pred[df_pred.UK_tok ==''].shape[0])
df_pred[df_pred.UK_tok ==''].head()

609


,RawText,Items,Problem,Solution,UK_tok
14,too_hot,,too_hot,,
17,too_hot,,too_hot,,
22,too_cold,,too_cold,,
50,reset_fault start_equipment,,,"reset, start",
55,heater not_work gate f.,"heater, gate",broken,,


In [19]:
# tag-empty? 
print(df_pred[(df_pred[['Items','Problem','Solution']]=='').all(axis=1)].shape[0])
df_pred[(df_pred[['Items','Problem','Solution']]=='').all(axis=1)].head()

43


,RawText,Items,Problem,Solution,UK_tok
40,cold a004 a005 fr bateman x5580,,,,"a005, fr, a004"
261,fabricate_metal access cover 28 x 38 1/2 hem f...,,,,"1/2, access, metal, hem, fabricated, fabricate..."
356,wet_carpet rm 114,,,,"wet_carpet, 114"
617,b349-mod-345b b367-mod-363b a368-mod-362b,,,,
632,cut_hole latch plate const_shop cut_hole latch...,,,,"plate const_shop, const_shop, plate const_shop..."
